In [316]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [317]:
mrd = pd.read_csv("bsm_data_train.csv", usecols=['MessageReceivedDate', 'ident_bag'])

In [318]:
mrd['MessageReceivedDate'] = pd.to_datetime(mrd['MessageReceivedDate']).dt.tz_localize(None)
mrd['MessageReceivedDate'] = mrd['MessageReceivedDate'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [319]:
mrd.set_index('MessageReceivedDate', inplace=True)

In [320]:
mrd.index = pd.to_datetime(mrd.index)

In [321]:
hourly_count = mrd.resample('H').count()

In [322]:
hourly_count.index = hourly_count.index.strftime('%Y-%m-%d %H:%M:%S')

In [323]:
hourly_count = hourly_count.drop(hourly_count.index[:645000])

In [324]:
hourly_count

,ident_bag
MessageReceivedDate,


In [304]:
slice_size = 300
slices = [hourly_count[i:i+slice_size] for i in range(0, len(hourly_count), slice_size)]

In [305]:
slices_count = len(slices)

In [281]:
plt.figure(figsize = (15,8))

for i in range(17):
    plt.plot(slices[i], color = 'steelblue')
    
    plt.xlabel('Дни', fontsize = 14)
    plt.ylabel('Количество пассажиров', fontsize = 14)
    plt.title('Перевозки пассажиров за 2023 год', fontsize = 16)
    
plt.show()    

IndexError: list index out of range

<Figure size 1500x800 with 0 Axes>

In [267]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [268]:
plt.figure(figsize=(15,8))

model = SARIMAX(hourly_count, order = (2, 0, 0), seasonal_order = (2, 1, 0, 50))
result = model.fit()
    
start = len(hourly_count)
end = len(hourly_count) + len(hourly_count) - 1
    
predictions = result.predict(start, end)
    
df_predictions = hourly_count.copy()
predictions.index = df_predictions.index
df_predictions['ident_bag'] = predictions

plt.plot(hourly_count, color = '#e99347')
plt.plot(df_predictions, linestyle='--', color='#77bb6a', )
    
plt.title("Тестовый прогноз количества пассажиров в разрезе группа авиакомпаний/терминал ")
plt.ylabel('Количество пассажиров')
plt.xlabel('Время')

plt.show()

MemoryError: Unable to allocate 1.76 GiB for an array with shape (152, 152, 10246) and data type float64

Exception ignored in: 'statsmodels.tsa.statespace._kalman_smoother.dKalmanSmoother.reset_filter_method'
Traceback (most recent call last):
  File "statsmodels\tsa\statespace\_kalman_smoother.pyx", line 988, in statsmodels.tsa.statespace._kalman_smoother.dKalmanSmoother.allocate_arrays
numpy.core._exceptions._ArrayMemoryError: Unable to allocate 1.76 GiB for an array with shape (152, 152, 10246) and data type float64


AttributeError: Memoryview is not initialized

<Figure size 1500x800 with 0 Axes>